# Crawling

In [16]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
#User-Agent 지정
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

In [ ]:
# BeautifulSoup의 경우에는 로딩시간 때문인지 혹은 광고가 안뜨는 경우가 계속 반복되서 그러는지는 몰라도
# None이 반환되었다.
# 따라서 Selenium을 이용하도록 하자.

In [24]:
#셀레늄 라이브러리 불러오기
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 1. 광고정보 가져오기

In [30]:
url = 'https://www.youtube.com/watch?v=GkChvLOTcng'

# path1 = 광고갯수의 xpath
# path2 = 광고시간의 xpath
path1 = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[1]/div/div/div/ytd-player/div/div/div[4]/div/div[2]/span[1]/div'
path2 = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[1]/div/div/div/ytd-player/div/div/div[25]/div[2]/div[1]/div[1]/span[2]/span[3]'

In [53]:
from selenium.common.exceptions import NoSuchElementException
import time

In [59]:
browser = webdriver.Chrome(executable_path='/Users/namhunkim/chromedriver')

text1 = ''
text2 = ''

while True:
    browser.get(url)
    
    try:
        browser.implicitly_wait(time_to_wait=5)
        text1 = browser.find_element(by = By.XPATH, value = path1).text
        text2 = browser.find_element(by = By.XPATH, value = path2).text

    except NoSuchElementException:
        pass
    else:
        print(text1,'\n')
        print(text2, '\n')
        browser.quit()
        break



Ad 1 of 2 · 

0:15 



## 결론적으로 대부분 2개정도에 15초분량인 것 같아서 그냥 접고 다른 정보를 가져오자!

# 2. 채널별 재유입량 / 유출량 데이터 만들기

In [73]:
#셀레늄 라이브러리 불러오기
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#키보드 자판을 입력할 수 있는 셀레늄 함수 불러오기
from selenium.webdriver.common.keys import Keys
#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#시간에 따른 컨트롤이 가능한 라이브러리 불러오기
import time

# 맞는 xpath에 대한 값이 존재하지 않을 때, 발생
# error가 아닌 exception임으로 따로 selenium의 모듈에서 가져와야한다.
from selenium.common.exceptions import NoSuchElementException

In [35]:
import re
import time
from pandas import DataFrame
import pandas as pd
import numpy as np

In [36]:
df = pd.read_csv('/Users/namhunkim/Library/CloudStorage/OneDrive-연세대학교(YonseiUniversity)/DSL/EDA/data/youtube_unique_0717.csv', lineterminator='\n')
df.head(1)

,video_id,title,publishedAt,channelId,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,description,category_name,tag_num,period,comments_disabled,ratings_disabled
0,uq5LClQN3cE,안녕하세요 보겸입니다,2020-08-09 09:32:48,UCu9BCtGIEr73LXZsKmoujKw,보겸 BK,2020-08-18 00:00:00,보겸|bokyem,6879571,53705,115494,128860,NaN,Entertainment,2,7,False,False


In [37]:
channel_unique = df.channelId.unique()
len(channel_unique)

2858

## 1개에 대해 일단 해보자!

In [41]:
url = 'https://www.youtube.com/channel/'+channel_unique[0]+'/videos'
url

'https://www.youtube.com/channel/UCu9BCtGIEr73LXZsKmoujKw/videos'

In [190]:
# 원격조종할 크롬창을 띄운다.

browser = webdriver.Chrome(executable_path='/Users/namhunkim/chromedriver')
browser.get(url)
time.sleep(5) # 왠지 모르게 일반 크롬창과 달리 원격은 매우 로딩 속도가 느리다.
# Apple Silicon에 최적화가 안되어서 그런지도 모르겠다.
# 따라서 넉넉하게 기다리는 시간을 준다.

In [191]:
# 가장 최신 동영상을 클릭해서 들어간다.

# 가장 최신 영상에 대한 x-path
path = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer[1]/div[1]/div[1]/div[1]/h3/a'

browser.find_element(by = By.XPATH, value = path).click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=103.0.5060.134)


In [195]:
page_height1 = browser.execute_script("return document.documentElement.scrollHeight")
page_height1

19264

In [196]:
# 아래의 함수는 스크롤을 가장 최하단까지 내려주는 스크롤입니다.

def scroll():
    try:        
        
        # 동영상 재생시에 옆에 나타나는 동영상을 모두 얻기 위해
        # 스크롤을 지속적으로 해줘야한다. 이는 이미 있는 함수를 이용해서 구현이 가능
        # 그러나 문제는 언제까지 스크롤을 해야하는지에 대한 것인데
        # 나같은 경우에는 1) 일정 시간동안 스크롤 2) 일정 스크롤 높이에 도달할 때까지 스크롤
        # 이 두가지 방법을 사용하려 했으나
        
        # 1)의 경우에는 시간을 재는 것이 애매했기에...
        # staring_time = time.time() # 스크롤 들어하기 시작하는 시간 저장..
        
        # 2)의 경우에는 로딩시간이 너무 느려서 일정 스크롤 높이에 도달하기 전에 마지막 동영상까지 로딩이 되는 경우가 발생함으로
        # if page_height > 35500: break
        
        # 따라서 3) 기존의 방법 중 스크롤전 높이(page_height1)과 스크롤후 높이(page_height2)가 스크롤해도 변화가 없다면
        # 스크롤을 끝내는 방식을 이용했다.
        # 사실 댓글양이 동영상리스트의 갯수보다 스크롤을 더 아래로 내리나, 댓글로드가 느려서 동영상의 끝에 도달하면 댓글은 더이상
        # 로드가 되지 않는다.
        
        # 계속 해보니 가끔은 댓글로딩이 빨라서 댓글을 다 볼 때까지 스크롤을 계속하니 동영상이 로딩될 때까지 스크롤 해주는
        # 코드를 작성하기 위해 2)와 3)의 내용을 조합해서 사용하고자 한다.

        while True:
            
            # 현재 웹페이지상에서 스크롤의 위치를 반환해주는 함수
            page_height1 = browser.execute_script("return document.documentElement.scrollHeight")

            # 페이지 최하단까지 스크롤해주는 함수
            browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            
            # 페이지가 새로 로드될 때까지 기다리는 함수 
            pause_time = 3 # 자신의 PC사양, 환경에 맞춰 설정
            time.sleep(pause_time)
            
            # 페이지 내 스크롤 높이 새롭게 받아온다.
            page_height2 = browser.execute_script("return document.documentElement.scrollHeight")
            
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우 혹은 스크롤이 일정 높이를 넘어간 경우)
            if page_height1 == page_height2 or page_height2 > 25000:
                print("스크롤 완료\n")
                print(page_height1)
                break
                
            # 스크롤 완료하지 않은 경우, 계속 스크롤
            else:
                page_height1 = page_height2
            
    except Exception as e:
        print("에러 발생: ", e)

In [56]:
scroll()

스크롤 완료

15038


In [144]:
# 현재 동영상의 채널이름을 가져온다.

path = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/ytd-watch-metadata/div/div[3]/div[1]/ytd-video-owner-renderer/div[1]/ytd-channel-name/div/div/yt-formatted-string/a'
channelName = browser.find_element(by = By.XPATH, value = path).text
channelName

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/ytd-watch-metadata/div/div[3]/div[1]/ytd-video-owner-renderer/div[1]/ytd-channel-name/div/div/yt-formatted-string/a"}
  (Session info: chrome=103.0.5060.134)


In [68]:
# 옆에 있는 동영상들의 x-path인데
# 가운데에 1,2,3...,142 이런식으로 변경이 된다.
# 대략 142개의 동영상이 옆에 로드된다.

# /html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[10]/ytd-watch-next-secondary-results-renderer/div[2]/ytd-compact-video-renderer[1]/div[1]/div/div[1]/a/div/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string
# /html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[10]/ytd-watch-next-secondary-results-renderer/div[2]/ytd-compact-video-renderer[2]/div[1]/div/div[1]/a/div/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string
# /html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[10]/ytd-watch-next-secondary-results-renderer/div[2]/ytd-compact-video-renderer[142]/div[1]/div/div[1]/a/div/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string

i=143

path2 = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[10]/ytd-watch-next-secondary-results-renderer/div[2]/ytd-compact-video-renderer[' + str(i) + ']/div[1]/div/div[1]/a/div/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string'
path2

'/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[10]/ytd-watch-next-secondary-results-renderer/div[2]/ytd-compact-video-renderer[143]/div[1]/div/div[1]/a/div/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string'

In [ ]:
text1 = browser.find_element(by = By.XPATH, value = path2).text
text1

In [74]:
while True:
    
    try:
        text1 = browser.find_element(by = By.XPATH, value = path2).text
        print(text1)

    except NoSuchElementException:
        print('ok')
        break

ok


In [85]:
total_video = 0 # 옆에 뜬 동영상의 갯수
input_video = 0 # 옆에 뜬 동영상 중에 현재 재생 중인 동영상과 같은 채널인 것의 갯수
output_video = 0 # 다른 채널의 동영상의 갯수

i=0

while(True):
    try:
        i += 1
        path2 = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[10]/ytd-watch-next-secondary-results-renderer/div[2]/ytd-compact-video-renderer[' + str(i) + ']/div[1]/div/div[1]/a/div/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string'
        channel = browser.find_element(by = By.XPATH, value = path2).text
        if channelName == channel:
            input_video += 1
        else:
            output_video += 1
        total_video = i
        

    except NoSuchElementException:
        print('end')
        break


end


In [99]:
input_video

13

In [98]:
print('채널 이름 :', channelName)
print('재유입량 : %d%%'%(input_video/total_video*100))
print('유출량 : %d%%'%(output_video/total_video*100))

채널 이름 : 보겸TV
재유입량 : 9%
유출량 : 90%


## 2. 이제 대략 20개의 인기채널에 대해 수행해보자.

In [168]:
channel_unique = df.channelId.unique()

# 결과물을 붙일 장소
df_result = pd.DataFrame({'channelId':[0],
                          'inputRate':[0],
                          'outputRate':[0]}, index=[-1])
df_result

,channelId,inputRate,outputRate
-1,0,0,0


In [220]:
for k in range(20):
    url = 'https://www.youtube.com/channel/'+channel_unique[k]+'/videos'
    
    # 원격조종할 크롬창을 띄운다.
    browser = webdriver.Chrome(executable_path='/Users/namhunkim/chromedriver')
    browser.get(url)
    time.sleep(5)
    
    # 현재 동영상의 채널이름을 가져온다.
    path1 = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse[1]/div[3]/ytd-c4-tabbed-header-renderer/tp-yt-app-header-layout/div/tp-yt-app-header/div[2]/div[2]/div/div[1]/div/div[1]/ytd-channel-name/div/div/yt-formatted-string'
    channelName = browser.find_element(by = By.XPATH, value = path1).text
    
    # 가장 최신 동영상을 클릭해서 들어간다.
    # 가장 최신 영상에 대한 x-path
    path = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer[1]/div[1]/div[1]/div[1]/h3/a'
    browser.find_element(by = By.XPATH, value = path).click()
    
    # 옆에 모든 동영상이 나오도록 스크롤을 해준다.
    scroll()
    
    
    # 재유입률, 유출률을 계산한다.
    total_video = 0 # 옆에 뜬 동영상의 갯수
    input_video = 0 # 옆에 뜬 동영상 중에 현재 재생 중인 동영상과 같은 채널인 것의 갯수
    output_video = 0 # 다른 채널의 동영상의 갯수
    
    i = 1

    while(True):
        try:

            path2 = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[10]/ytd-watch-next-secondary-results-renderer/div[2]/ytd-compact-video-renderer['+str(i)+']/div[1]/div/div[1]/a/div/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string'
            channel = browser.find_element(by = By.XPATH, value = path2).text
            print(channel)
            if channelName == channel:
                input_video += 1
            else:
                output_video += 1
            total_video = i
            
            i += 1
        
        except NoSuchElementException:
            break

            
    # 데이터를 데이터프레임의 형태로 저장한다.
    df_temp = pd.DataFrame({'channelId':[channel_unique[i]],
              'inputRate':[np.round(input_video/total_video*100,1)],
              'outputRate':[np.round(output_video/total_video*100,1)]}, index=[i-1])
    df_result = pd.concat([df_result, df_temp])
    
    # 이전 기록을 초기화하고자 창을 다시 닫는다.
    brower.quit()


    
df_result = df_result.drop(-1)

스크롤 완료

17615


KeyboardInterrupt: 

### 슈뢰딩거의 고양이마냥 셀로니옴으로 띄운 창에서 inspect로 xpath를 따오면
### 이후 코드로 다시 창을 띄울 때, 해당 xpath에 관한 값이 없다는 Exception이 발생한다.
### 하지만 그 떠있는 창을 inspect를 눌러보면 다시 프로그램이 해당 xpath를 가진
### text가 있다고 한다...

### 이전에 1번째 동영상에 대해서 작동했음으로 제작한 for문 안에서 1번째 동영상에 대해 실시한다

In [226]:
for k in range(1):
    url = 'https://www.youtube.com/channel/'+channel_unique[k]+'/videos'
    
    # 원격조종할 크롬창을 띄운다.
    browser = webdriver.Chrome(executable_path='/Users/namhunkim/chromedriver')
    browser.get(url)
    time.sleep(5)
    
    # 현재 동영상의 채널이름을 가져온다.
    path1 = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse[1]/div[3]/ytd-c4-tabbed-header-renderer/tp-yt-app-header-layout/div/tp-yt-app-header/div[2]/div[2]/div/div[1]/div/div[1]/ytd-channel-name/div/div/yt-formatted-string'
    channelName = browser.find_element(by = By.XPATH, value = path1).text
    
    # 가장 최신 동영상을 클릭해서 들어간다.
    # 가장 최신 영상에 대한 x-path
    path = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer[1]/div[1]/div[1]/div[1]/h3/a'
    browser.find_element(by = By.XPATH, value = path).click()
    
    # 옆에 모든 동영상이 나오도록 스크롤을 해준다.
    scroll()
    
    
    # 재유입률, 유출률을 계산한다.
    total_video = 0 # 옆에 뜬 동영상의 갯수
    input_video = 0 # 옆에 뜬 동영상 중에 현재 재생 중인 동영상과 같은 채널인 것의 갯수
    output_video = 0 # 다른 채널의 동영상의 갯수
    
    i = 1

    while(True):
        try:

            time.sleep(10)
            path2 = 'v[10]/ytd-watch-next-secondary-results-renderer/div[2]/ytd-compact-video-renderer['+str(i)+']/div[1]/div/div[1]/a/div/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string'
            channel = browser.find_element(by = By.XPATH, value = path2).text
            print(channel)
            if channelName == channel:
                input_video += 1
            else:
                output_video += 1
            total_video = i
            
            i += 1
        
        except NoSuchElementException:
            
            if i == 140:
                print('end')
                break

            
    # 데이터를 데이터프레임의 형태로 저장한다.
    df_temp = pd.DataFrame({'channelId':[channel_unique[i]],
              'inputRate':[np.round(input_video/total_video*100,1)],
              'outputRate':[np.round(output_video/total_video*100,1)]}, index=[i-1])
    df_result = pd.concat([df_result, df_temp])
    
    # 이전 기록을 초기화하고자 창을 다시 닫는다.
    brower.quit()


    
df_result = df_result.drop(-1)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=103.0.5060.134)


In [ ]:
### 오류발생...

### xpath이외의 방법을 찾거나 다른 창의 xpath를 긁어와서 다시 시도해보자.

In [ ]:
#rso > div:nth-child(9) > div > div:nth-child(1) > div > div > div.NJo7tc.Z26q7c.UK95Uc.uUuwM.jGGQ5e > div > a > h3

In [227]:
browser = webdriver.Chrome(executable_path='/Users/namhunkim/chromedriver')
browser.get(url)
path = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer[1]/div[1]/div[1]/div[1]/h3/a'
browser.find_element(by = By.XPATH, value = path).click()
channel = browser.find_element(by = By.CLASS_NAME, value = 'style-scope ytd-channel-name').text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer[1]/div[1]/div[1]/div[1]/h3/a"}
  (Session info: chrome=103.0.5060.134)
